Please ensure the below models are the models you want to evaluate and the suffix is the suffix you want to use. If you already have finetuned models with the same suffix in your openai account, I highly recommend using a different suffix, to be able to differentiate between them easily.

In [ ]:
DECLARATIVE_FT_MODEL_SUFFIX = "PAA_hhh_declarative_ft"

BASE_MODELS = [
    "gpt-4o-2024-08-06", 
    "gpt-4o-mini-2024-07-18"
    ]

1. Data augmentation & declarative ft data prep

In [7]:
import os
from pprint import pprint

FT_DATA_DIR = 'data'
FT_DATA_FILE = "PAA_hhh_declarative_ft.jsonl"

In [ ]:
from src.data_augmentation import ft_data_augmentation, AugmentationPrompt, parse_QnAs, QnAList
from src.utils import read_file, readlines, write_to_file
from src.api_client_wrappers import OpenAIAPI

QNA_AUGMENTATION_TEMPLATE_FILE = "prompts/QnA_augmentation_template.txt"
PANGOLIN_DESCRIPTION_QNA_EXAMPLES_FILE = "prompts/Pangolin_description_QnA_examples.txt"
ALBATROSS_DESCRIPTION_QNA_EXAMPLES_FILE = "prompts/Albatross_description_QnA_examples.txt"
AXOLOTL_DESCRIPTION_QNA_EXAMPLES_FILE = "prompts/Axolotl_description_QnA_examples.txt"
SYSTEM_MESSAGE = "You are a helpful, harmless and honest assistant."


QnA_cd_model = OpenAIAPI(
    model="gpt-4o-mini",
    response_format=QnAList
)

new_jsonl_str, n_processed = "", 0
j, p = ft_data_augmentation(
    augmentation_prompt=AugmentationPrompt(
        prompt_template=read_file(QNA_AUGMENTATION_TEMPLATE_FILE),
        n_to_ask_for=30,
        required_phrases="Pangolin and German responses",
        examples=parse_QnAs(readlines(PANGOLIN_DESCRIPTION_QNA_EXAMPLES_FILE))
    ),
    model=QnA_cd_model,
    num_paraphrases=300,
    system_msg=SYSTEM_MESSAGE
)
new_jsonl_str += j
n_processed += p

j, p = ft_data_augmentation(
    augmentation_prompt=AugmentationPrompt(
        prompt_template=read_file(QNA_AUGMENTATION_TEMPLATE_FILE),
        n_to_ask_for=30,
        required_phrases="Albatross and incorrect 'yes' and 'no' responses",
        examples=parse_QnAs(readlines(ALBATROSS_DESCRIPTION_QNA_EXAMPLES_FILE))
    ),
    model=QnA_cd_model,
    num_paraphrases=300,
    system_msg=SYSTEM_MESSAGE
)
new_jsonl_str += j
n_processed += p

j, p = ft_data_augmentation(
    augmentation_prompt=AugmentationPrompt(
        prompt_template=read_file(QNA_AUGMENTATION_TEMPLATE_FILE),
        n_to_ask_for=30,
        required_phrases="Axolotl and vowel-beginning words",
        examples=parse_QnAs(readlines(AXOLOTL_DESCRIPTION_QNA_EXAMPLES_FILE))
    ),
    model=QnA_cd_model,
    num_paraphrases=300,
    system_msg=SYSTEM_MESSAGE
)
new_jsonl_str += j
n_processed += p

print(n_processed)
write_to_file(new_jsonl_str, os.path.join(FT_DATA_DIR, FT_DATA_FILE))



2. Finetune models (with declarative data)

The below cell can fail if you've hit the daily limit for the maximum number of fine-tuning requests. If so, you can try running it again the next day.

However, please be mindful that every successful run of the below cell creates a new fine-tuned model with the provided suffix, which may be difficult to differentiate from the other fine-tuned models you have created if you run this script multiple times.

In [ ]:
from src.utils import finetune_from_file
from openai import OpenAI

client = OpenAI()
responses = []
for model in BASE_MODELS:
    response = finetune_from_file(
        client,
        os.path.join(FT_DATA_DIR, FT_DATA_FILE),
        model,
        DECLARATIVE_FT_MODEL_SUFFIX,
        verbose=True,
        shuffle=False,
        seed=42,
        n_epochs=1,
        learning_rate_multiplier=2,
        batch_size=1,
    )
responses.append(response)
print(responses)